# Generate FORTRAN static modules with OTI or Multidual algebras.
```
Author: Mauricio Aristizabal
Date last modified: Jan 13 2022
Initial date: Jan 13 2022
Contact: mauriaristi@gmail.com
```

This Notebook lets you generate Fortran modules that implements static versions of the library.

They implement the so called semi-sparse implementation to avoid computational overheads.


In [2]:
import sys
import os
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.core   as coti
from pyoti.fmod_writer import writer
# from pyoti.whereotilib import getpath

In [3]:
nbases = 1
order  = 1
w = writer(nbases, order)

AttributeError: 'writer' object has no attribute 'endl'